In [13]:
async def detect():
  pass

In [14]:
context = {
  'detect': detect
}

In [66]:
import ast
import inspect
import re
from typing import Any, Dict


def preprocess_command(command: str, context: Dict[str, Any]) -> str:
    def replace_var(var):
        # If it is a defined coroutine function (already in context)
        if var in context and inspect.iscoroutinefunction(context[var]):
            return f"await __context['{var}']"

        return f"__context['{var}']"
      
    processed_command = command
    while True:
        # Re-parse the command to get fresh AST after each replacement
        node_iter = ast.walk(ast.parse(processed_command, mode="exec"))
        replacements = []

        for node in node_iter:
            if isinstance(node, ast.Name) and node.id != "__context":
                start, end = node.col_offset, node.end_col_offset
                replacements.append((start, end, replace_var(processed_command[start:end])))

        if not replacements:
            break

        # Apply replacements in reverse order to avoid messing up indices
        for start, end, replacement in sorted(replacements, reverse=True):
            processed_command = processed_command[:start] + replacement + processed_command[end:]

    return processed_command

      

In [67]:
preprocess_command("rose = detect(image, prompt='rose')", context)

"__context['rose'] = await __context['detect'](__context['image'], prompt='rose')"